In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
import os

In [3]:
import pickle

In [4]:
import pylangacq as pla

In [5]:
import random

Load Targets

In [6]:
# y_cookie = torch.load('cookie_data_targets.pt')

In [6]:
y_cookie = torch.load('cookie_target_dict.pt')

In [8]:
len(y_cookie)

552

In [9]:
# y_sentence = torch.load('sentence_data_targets.pt')

In [7]:
y_sentence = torch.load('sentence_target_dict.pt')

In [11]:
len(y_sentence)

240

Load Samples

In [8]:
X_cookie_filenames = torch.load('cookie_fnames_dict.pt')

In [13]:
len(X_cookie_filenames)

552

In [9]:
X_sent_filenames = torch.load('sentence_fnames_dict.pt')

In [15]:
len(X_sent_filenames)

240

In [10]:
num_samples = len(X_cookie_filenames) + len(X_sent_filenames)

In [17]:
num_samples

792

In [11]:
X_cookie_reader = pla.read_chat('./Pitt/*/cookie/*.cha')
X_sentence_reader = pla.read_chat('./Pitt/*/sentence/*.cha')

-----------------------------------

Missing data -- targets

group missing or input error st no mean to replace with and left untouched in target generation

replace with closest analog as per means generated from target generation:

cookie {'Control': 30, 'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 20, 'Probable': 19, 'Other': 24}

sentence {'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 21, 'Control': 30, 'Probable': 19, 'Other': 24}

In [19]:
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])

In [21]:
target_674 = X_sentence_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/236-0.cha']['Participants']['PAR']['group']

In [143]:
target_674

''

In [144]:
group_674

'Dementia'

In [154]:
# Replace Dementia with Possible mean (as per diagnostic codes in Pitt-data.xlxs and accompanying Pitt-readme.pdf)
y_sentence[674] = 21

In [139]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha
219 
/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha
511 possibleAD


In [146]:
target_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['education']
group_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['group']

In [147]:
target_219

''

In [148]:
group_219

''

In [156]:
# Replace '' with Control mean (as per folder and dx codes in .xlxs)
y_cookie[219] = 30

In [149]:
target_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['education']
group_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['group']

In [150]:
target_511

''

In [151]:
group_511

'possibleAD'

In [158]:
# Replace possibleAD with PossibleAD mean
y_cookie[511] = 20

In [159]:
# confirm all targets filled now 
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])


In [160]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

In [161]:
# torch.save(y_cookie,'cookie_target_dict.pt')

In [162]:
# torch.save(y_sentence,'sentence_target_dict.pt')

---------------------------------------------------------------

Load Vocab Embeddings

In [12]:
glove_path = './PretrainedWordEmb/'

In [13]:
vocab_emb = pickle.load(open(f'{glove_path}/addb.vocab_emb.glove.42B.300.pkl', 'rb'))

In [33]:
len(vocab_emb)

2188

In [14]:
pos_dict = torch.load('pos_dict.pt')

In [267]:
num_samples

792

In [59]:
# valid_ix = random.sample(range(num_samples),int(.20*num_samples))

In [61]:
# train_ix = [i for i in range(num_samples) if i not in valid_ix]

In [63]:
len(valid_ix)

158

In [62]:
len(train_ix)

634

In [65]:
# torch.save(valid_ix,'valid_ix.pt')

In [67]:
# torch.save(train_ix,'train_ix.pt')

In [72]:
valid_ix = torch.load('valid_ix.pt')

In [73]:
len(valid_ix)

158

In [74]:
train_ix = torch.load('train_ix.pt')

In [75]:
len(train_ix)

634

In [62]:
def get_item(ix=None,valid=False):
    
    # first should be...all zeros: each index represents greater than .. but you cant have a zero score..
    def num_vectorize(t):

        v = torch.zeros([29])
        v[:t-1] = 1

        return v.double()
    
    ix = random.randint(1,num_samples-1) if ix is None else ix
    
    #index, filename
    if ix in X_cookie_filenames.keys():
        _file = (ix,X_cookie_filenames[ix])
        _reader = X_cookie_reader
        _targetdict = y_cookie
        
    else:
        _file = (ix,X_sent_filenames[ix])
        _reader = X_sentence_reader
        _targetdict = y_sentence
        
    file,data,targetdict = (_file,_reader,_targetdict)
    
    embeddings = []
    targets = []

    embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file[1]] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file[1]] for (tokensraw,pos,tokenstem,dependency) in utterance])]
    target = targetdict[ix]

    for tkn in embedding:

        tkn[1][tkn[2]] = 1
#     print(target)   
    if valid: return [torch.cat((tkn[0],tkn[1])) for tkn in embedding], torch.tensor(target)
    return [torch.cat((tkn[0],tkn[1])) for tkn in embedding], num_vectorize(target)


In [70]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [288]:
def pad_minibatch(minib):
    
    batchsize = len(minib)
    
    seq_lens = [len(emb) for emb in minib]
    max_len = max(seq_lens)
    
    # input of shape (seq_len, batch, input_size): tensor containing the features of the input sequence
    # https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
#     seq_tensor = torch.zeros((batchsize, max_len, EMBEDDING_SIZE)).double()
    seq_tensor = torch.zeros((max_len, batchsize, EMBEDDING_SIZE)).double()
    
                  
    for i, (seq,length) in enumerate( zip(minib,seq_lens) ):
        for wi,word in enumerate(seq):
#             seq_tensor[i,wi] = word
            seq_tensor[wi,i] = word
    # mod to sort at the start?
    seq_tensor = nn.utils.rnn.pack_padded_sequence(seq_tensor,lengths=seq_lens,enforce_sorted=False)
    return seq_tensor

In [306]:
#pad with zeros -- maybe
# end with eos tensor, tag

def get_minibatch(batchsize=1, ix=None):
    
    
    def num_vectorize(targets):
        vectors = []
        for i in targets:
            v = torch.zeros([30])
            v[:i] = 1
            vectors.append(v)
        return vectors
    
    
    minibatch_ix = random.sample(range(num_samples),batchsize) if ix is None else ix 
    
    #index, filename
    cookie_files = [(i,X_cookie_filenames[i]) for i in minibatch_ix if i in X_cookie_filenames.keys()]
    sent_files = [(i,X_sent_filenames[i] )for i in minibatch_ix if i in X_sent_filenames.keys()]
    
    embeddings = []
    targets = []

    for corpus,data,targetdict in [(cookie_files,X_cookie_reader,y_cookie),(sent_files,X_sentence_reader,y_sentence)]: 
        for file_ix,file in corpus:
#             print(file_ix, file)
#             print('Words',[tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])
#             print('Words_len',len([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance]))
            embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])]
            target = targetdict[file_ix]
            
            for tkn in embedding:

                tkn[1][tkn[2]] = 1

#                 embeddings.append(torch.cat((tkn[0],tkn[1])))
            
            embeddings.append([torch.cat((tkn[0],tkn[1])) for tkn in embedding])


#             embeddings.append(embedding)
            targets.append(target)
#     print(targets)       
#     return embeddings, torch.tensor(targets), minibatch_ix
#     return embeddings, torch.tensor(targets)

# return target vals for accuracy 
    return embeddings, num_vectorize(targets), torch.tensor(targets)

In [357]:
# https://towardsdatascience.com/simple-trick-to-train-an-ordinal-regression-with-any-classifier-6911183d2a3c
# validate -- accuracy 
def out_to_score_proba(yhat_vector):
    # 29-dim sized vector
    proba_vector = torch.zeros([30])
    # no zero-score
    proba_vector[0] = 0. 
    # probability == 1 , 1-P(y>1)   
    proba_vector[1] = 1-yhat_vector[0]
    
    for i in range(2,len(yhat_vector)):
        proba_vector[i] = yhat_vector[i-1] - yhat_vector[i]
        
    proba_vector[-1] = yhat_vector[-1] 
    
#     print(proba_vector)
    return torch.argmax(proba_vector)


In [28]:
out_to_score_proba(t)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.])

In [381]:
# shouldnt be on y but on targets ...
def accuracy_distansum(yhat_tensor,y_tensor):
    return (torch.abs(y - yhat).sum())

In [344]:
# fix to rate
def accuracy(yhat_tensor,y_tensor):
    return torch.stack([ya==yb for (ya,yb) in zip(yhat_tensor,y_tensor)]).sum()
    return ((y_hat == y_tensor).sum())/len(y_tensor)

In [232]:
# accuracy measurement
# return summation -- doesnt perfectly handle skipped values
# np.where(vector)[0][-1] https://stackoverflow.com/questions/38375401/neural-network-ordinal-classification-for-age
def out_to_score_summation(vector):
    # zeros array 
    
    return vector.sum() + 1
    
    if len(np.where(vector)[0]) == 0: return 0
#     print ([i+2 for i in np.where(vector)])
    print(np.where(vector))
    candidate = (np.where(vector)[0][-1]) 
    
    # handling intermediate zeros...maybe not optimal
    if np.where(vector)[0][-2] < (candidate - 1): candidate = (np.where(vector)[0][-2] + np.where(vector)[0][-1])/2
        
    return candidate + 1
    # alternative
    return vector.sum() + 1
#     len(np.where(t)[0])

In [218]:
# batch of all samples:  longest sample length (for padding, if minibatch)
max([len(emb) for emb in e])

546

In [199]:
# all words same len of emb size 371 
set([len(emb[i]) for emb in e for i in range(len(emb))])

{371}

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

Seq2Seq -- with attention

2 layer rnn..

In [30]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

371

In [31]:
HIDDEN_SIZE = 128
# EMBEDDING_SIZE = 371
# with EOS pos
EMBEDDING_SIZE = len(vocab_emb['I']) + len(pos_dict)

In [268]:
# max([len(emb) for emb in e])

546

In [32]:
# change if aadd in EOS, pos to 547
MAX_LENGTH = 546

In [33]:
VOCAB_SIZE = len(vocab_emb)

In [192]:
BATCH_SIZE = 32

In [102]:
# pos_dict['EOS'] = len(pos_dict)

In [104]:
# EOS tag
# vocab_emb['EOS'] = torch.rand([300],dtype=torch.float64)

In [322]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size=MAX_LENGTH, emb_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE):
        super(EncoderRNN, self).__init__()
        
        self.emb_size = emb_size
        self.hidden_size = hidden_size

        # what is the input size fo self.embedding...? and is self.embedding necessary
#         self.embedding = nn.Embedding(input_size, emb_size)
        
        
        
#         self.gru = nn.GRU(hidden_size, hidden_size)
#         self.gru = nn.GRU(input_size=self.emb_size, hidden_size=self.hidden_size,num_layers=2)
# batch_first – If True, then the input and output tensors are provided as (batch, seq, feature) https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
        self.gru = nn.GRU(input_size=self.emb_size,hidden_size=self.hidden_size,num_layers=2).double()
#         # each output node Oi of our neural network
# uses a standard sigmoid function 1
# 1+e−zi
# , without including
# the outputs from other nodes, as shown in Figure 1. Output
# node Oi
# is used to estimate the probability oi
# that a data
# point belongs to category i independently, without subjecting
# to normalization as traditional neural networks do. Thus,
# for a data point x of category k, the target vector is
# (1, , 1, .., 1, 0, 0, 0), in which the first k elements is 1 and
# others 0
# http://orca.st.usm.edu/~zwang/files/rank.pdf A Neural Network Approach to Ordinal Regression
    
        self.fc = nn.Linear(self.hidden_size*2,30).double()
        
        self.activation = nn.Sigmoid()
#         self.activations_list = [nn.Sigmoid() for i in range(30)]

    def forward(self, input, hidden):
        # confirm dims here...
#         embedded = self.embedding(input).view(1, 1, -1)
        
        embedded = input.double()
#         embedded = torch.cat(input).view(len(input), batch_size, -1).double()
        
#         output = embedded
        
#         output, hidden = self.gru(self.emb_size, self.hidden_size)
#         output, hidden = self.gru(output)
        gru_out, _ = self.gru(embedded,hidden)
        
        hid_out = torch.cat((_[-2,:,:], _[-1,:,:]), dim = 1)
#         print(gru_out.size(),_.size())
    
#         lstm_out, _ = self.lstm(embedding.view(len(sentence), -1))
        out = self.fc(hid_out)
        print(out.size())
        
#         out = torch.tensor([a(o) for (a,o) in zip(self.activations_list,out[0])],requires_grad=True).double()
        out - self.activation(out)
        
        # any further processing -- int? softmax...? 
#         return out
        return out, _
        return torch.nn.utils.rnn.pad_packed_sequence(out), _

    def initHidden(self,batch_size=BATCH_SIZE):
        # h_0 of shape (num_layers * num_directions, batch, hidden_size): tensor containing the initial hidden state for each element in the batch. If the LSTM is bidirectional, num_directions should be 2
        return torch.zeros(2, batch_size, self.hidden_size, device=device).double()
        


In [315]:
# The
# cost function for a data point x can be relative entropy
# or square error between the target vector and the output
# vector
# loss = nn.BCELoss()
loss_func = nn.MSELoss()

# TODO minibatch + pad. For starters, stochastic gd..

In [365]:
encoder = EncoderRNN()

In [366]:
optimizer = torch.optim.Adam(encoder.parameters(), lr=1e-2)

In [318]:
_hidden = encoder.initHidden()

In [343]:
torch.stack([ya==yb for (ya,yb) in zip(y,yhat)]).sum()

tensor(0)

In [337]:
y.size()

torch.Size([32, 30])

In [338]:
yhat.size()

torch.Size([32, 30])

In [328]:
loss_func

MSELoss()

In [351]:
yhat.size()

torch.Size([32, 30])

In [352]:
sum([1 for h in yhat])

32

In [358]:
out_to_score_proba(yhat[0])

tensor(29)

In [362]:
torch.tensor([out_to_score_proba(hat) for hat in yhat])

tensor([29, 29, 29, 29, 28, 29, 29, 24, 29, 29, 29, 29, 29, 29, 29, 29, 28, 19,
        19, 29, 29,  7, 28, 19, 28, 19, 19, 28, 12, 17, 19, 28])

In [363]:
targets

tensor([30, 29, 24, 29, 25, 30, 30, 18, 30, 30, 29, 18, 16, 28,  3, 29, 18, 11,
        19, 30, 30,  7, 29, 24, 21, 11, 21, 27, 10, 19, 11, 28])

In [367]:
encoder.train()
# _hidden = encoder.initHidden()


for epoch in range(10):
    
    # modify for ordering -- benefit here with shuffling at ea epoch
    random.shuffle(train_ix)
    
    print('Epoch',epoch)
    step = 0
    
    # in future, mix up epochs 
    for i in range(19):
        
        step += 1
        optimizer.zero_grad()
        
        x,y,targets = get_minibatch(batchsize=BATCH_SIZE)
        x = pad_minibatch(x)
        
        y = torch.stack(y).double()
        
        _hidden = encoder.initHidden(batch_size=BATCH_SIZE)

        yhat, _hidden = encoder(x, _hidden)

        loss = loss_func(yhat,y)

        loss.backward()
#         loss.backward(retain_graph=True)
        
        optimizer.step()

       
        print('step',step)
        print('loss',loss)

        yhat_tensor = torch.tensor([out_to_score_proba(hat) for hat in yhat])
        
        print('accuracy_distance',accuracy_distansum(yhat_tensor=yhat_tensor,y_tensor=targets))
        print('accuracy',accuracy(yhat_tensor=yhat_tensor,y_tensor=targets))
    

    
    
    

Epoch 0
torch.Size([32, 30])
step 1
loss tensor(0.8583, dtype=torch.float64, grad_fn=<MseLossBackward>)
accuracy_distance tensor(25.0028, dtype=torch.float64, grad_fn=<DivBackward0>)
accuracy tensor(0)
torch.Size([32, 30])
step 2
loss tensor(0.1608, dtype=torch.float64, grad_fn=<MseLossBackward>)
accuracy_distance tensor(9.5660, dtype=torch.float64, grad_fn=<DivBackward0>)
accuracy tensor(1)
torch.Size([32, 30])
step 3
loss tensor(0.4708, dtype=torch.float64, grad_fn=<MseLossBackward>)
accuracy_distance tensor(17.2547, dtype=torch.float64, grad_fn=<DivBackward0>)
accuracy tensor(0)
torch.Size([32, 30])
step 4
loss tensor(1.3132, dtype=torch.float64, grad_fn=<MseLossBackward>)
accuracy_distance tensor(31.5136, dtype=torch.float64, grad_fn=<DivBackward0>)
accuracy tensor(0)
torch.Size([32, 30])
step 5
loss tensor(0.2835, dtype=torch.float64, grad_fn=<MseLossBackward>)
accuracy_distance tensor(12.1988, dtype=torch.float64, grad_fn=<DivBackward0>)
accuracy tensor(0)
torch.Size([32, 30])
st

In [368]:
torch.save(encoder,'encoderv1.pt')

/Users/renee/opt/anaconda3/envs/dl1010/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type EncoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/renee/opt/anaconda3/envs/dl1010/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GRU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/renee/opt/anaconda3/envs/dl1010/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/renee/opt/anaconda3/envs/dl1010/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sigmoid. It won't be checked 

In [369]:
encoder.eval()

x_valid,y_valid,targets_valid = get_minibatch(ix=valid_ix)

x_valid = pad_minibatch(x_valid)
        
y_valid = torch.stack(y_valid).double()

_hidden = encoder.initHidden(batch_size=len(valid_ix))

yhat_valid, _hidden = encoder(x_valid, _hidden)


torch.Size([158, 30])


In [370]:
torch.tensor([out_to_score_proba(hat) for hat in yhat_valid])

tensor([29, 18, 29, 29, 26, 28, 23, 29, 22, 17, 20, 20, 25, 29, 28, 28, 29, 29,
        19, 16, 29,  7, 28, 29, 28, 28, 28, 12, 29, 12, 29, 25, 19, 18, 28, 26,
        27, 29, 20, 17, 29, 26, 18, 29, 19, 20, 20, 29, 28, 28, 29, 19, 28, 20,
        22, 29, 19, 29, 29, 29, 29, 22, 19, 27, 28, 29, 29, 22, 29, 29, 29, 28,
        29, 20, 29, 29, 29,  8, 29, 20, 29, 29, 29, 16, 28, 22, 29, 28, 16, 29,
        29, 19,  3, 29, 19, 29, 29, 20, 29, 20, 29, 19, 12, 29, 20, 23, 13, 20,
        23, 29, 29, 19, 20, 19, 19, 19, 27, 29, 10, 20, 18, 18, 24, 26, 16, 20,
        18, 17, 28, 28, 29, 20, 27, 20, 29, 19, 20, 28, 29, 22, 18, 20, 20, 19,
        19, 28, 28, 28, 26, 19, 24, 22, 18, 18, 20, 28, 25, 27])

In [371]:
targets_valid

tensor([30, 18, 29, 28, 26, 27, 23, 30, 19, 17, 20, 17, 25, 30, 23, 29, 27, 30,
        19, 16, 30,  7, 28, 28, 28, 28, 25, 12, 30, 18, 30, 25, 15, 18, 14, 26,
        27, 30, 20, 17, 30, 26, 19, 27, 17, 20, 20, 29, 27, 29, 30, 30, 30, 20,
        22, 29, 19, 30, 30, 30, 30, 22, 17, 27, 27, 30, 30, 23, 30, 30, 19, 25,
        30, 19, 30, 30, 30,  8, 30, 22, 30, 30, 28, 16, 19, 22, 24, 28, 16, 30,
        30, 19, 17, 27, 19, 30, 30, 21, 28, 19, 29, 19, 12, 28, 20, 23, 13, 20,
        23, 27, 23, 19, 20, 19, 19, 19, 27, 16, 10, 20, 17, 24, 24, 23, 16, 20,
        18, 17, 19, 28, 29, 20, 27, 20, 23, 16, 20, 23, 27, 22, 19, 20, 20, 19,
        19, 28, 28, 28, 26, 19, 24, 22, 20, 18, 23, 27, 25, 25])

In [376]:
sum(torch.tensor([out_to_score_proba(hat) for hat in yhat_valid]) == targets_valid)

tensor(78)

In [377]:
len(targets_valid)

158

In [378]:
78/158

0.4936708860759494

In [382]:
accuracy_distansum(y_tensor=targets_valid,yhat_tensor=torch.tensor([out_to_score_proba(hat) for hat in yhat_valid]))

tensor(106.8160, dtype=torch.float64, grad_fn=<SumBackward0>)

In [383]:
106/158

0.6708860759493671

In [290]:
x,y = get_minibatch(batchsize=1)

In [287]:
torch.cat(x[0]).view(len(x[0]),1, -1).shape

torch.Size([55, 1, 371])

In [251]:
# need to be able to pass any/all into net
# encoder = EncoderRNN(len(x[0])).to(device)

In [293]:
_hidden.size()

torch.Size([2, 1, 128])